# Data Preparation

## Импорт данных

In [1]:
import numpy as np
import pandas as pd

Набор данных содержит все доступные данные по более чем 800 000 потребительским кредитам, выданным с 2007 по 2015 год Lending Club: крупной американской компанией по одноранговому кредитованию. Существует несколько различных версий этого набора данных. Я использовала версию, доступную на kaggle.com.

In [ ]:
Заггрзуить данные можно при помощи следующего кода:
def load_data(filename):
    try:
        df=pd.read_csv(filename)
        return df
    except FileNotFoundError:
        print("{} does not exist".format(filename))
filename = "./loan_data_2007_2014.csv"
data = load_data(filename=filename)

In [6]:
Если мы хотим чтобы отображались все названия стоблцов, то применяем следующую формулу: pd.options.display.max_columns = None


In [ ]:
С помощью data.head() можно увидеть первые строки (информацию) датафрэйма
data.tail() показывает последние строки (информацию) датафрэйма

In [ ]:
Чтобы рассмотреть все названия столбцов в наборе данных, используем: data.columns.values

In [ ]:
data.info() предоставляет информацию о типе данных, наличии нулевых значений. Таким образом показывая общую информацию об используемых данных.

## Предобработка

Предобработка данных предусматривает сведение данных, конкретного стоблца или строки, к определенному формату.
Например, столбец "emp_length" означает работает ли кандидат, который хотел бы получить разрещение на кредит. И если работает то какой период времени (меньше 1 года, 1 год, 8 лет и т.д)

In [ ]:
Посмотреть все уникальные значения можно при помощи unique(), который отражает список.
Если в данных присутствуют такиие символы как "\+", "n/a", "years", мы заменяем их на 0 (n/a или меньше 1 года продолжительность работы),а str значеня становятся пустыми (к примеру, "years" становится " ").
Замена значений осуществляется при помощи replace
Далее видоизмененные str значения перевод в числовые, используя pd.to_numeric(data['emp_length'])
С помощью type(data['emp_length'][0]) можно проверить формат данных. Тепеь это numpy.float64
Такие же действия осуществляются с другими столбцами.
Если это столбец (дата (год, месяц, день)). То мы сначала перевод в формат pd.to_datetime и затем указываем как будет выгдялеть дата (формат, к примеру, "%B-%Y" (название месяца и года))

# Предварительная обработка нескольких дискретных переменных

In [37]:
pd.get_dummies(loan_data['grade'])

,A,B,C,D,E,F,G
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
466280,0,0,1,0,0,0,0
466281,0,0,0,1,0,0,0
466282,0,0,0,1,0,0,0
466283,1,0,0,0,0,0,0


У нас есть переменная "Grade", которая определяет категорию, к которой относятся кандидаты (A-высокий шанс на получения кредита и G-низкий.
С помощью pd.get_dummies мы получаем семь фиктивных переменных, соответствующих точному числу категорий переменной grade.
Существует только одна единица, а остальные значения равны нулям.
Значение единицы показывает, к какой категории переменной относится наблюдение.
функция get dummies создает столько фиктивных переменных, сколько категорий по умолчанию.
При этом вновь созданные переменные имеют те же имена, что и категории.


# Поиск и обработка пропущенных значений

In [ ]:
Количество нулевых значений можно посмотреть через data.isnull()
Заполнить нулевые значений можно через fillna(0, inplace = True). Зависит от столбца и какими значениями необходимо заполнить

К примеру, для годового дохода («annual_inc») заменить недостающие значения средним значением всех элементов переменной.

In [49]:
Это все нужные переменные
data['mths_since_earliest_cr_line']
data['acc_now_delinq']
data['total_acc']
data['pub_rec']
data['open_acc']
data['inq_last_6mths']
data['delinq_2yrs']
data['emp_length_int']

# PD (вероятность дефолта) модель

Каждый кредит имеет один из этих статусов: полностью погашенных ( в течение 31-120 дней), опоздавший от 16 до 30 дней, полностью оплаченный, и те что не соответствуют статусу кредитной политики.
Очевидно, что счета, которые были полностью оплачены, не были дефолтными.Они хороши. С другой стороны, списанные счета определенно дефолтировали. Поэтому они плохие (charged off)

In [51]:
К примеру
data['loan_status'].value_counts()

Current                                                224226
Fully Paid                                             184739
Charged Off                                             42475
Late (31-120 days)                                       6900
In Grace Period                                          3146
Does not meet the credit policy. Status:Fully Paid       1988
Late (16-30 days)                                        1218
Default                                                   832
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

Вот как будет работать логистическая регрессия:

Сначала мы оценим взаимосвязь между зависимой переменной и независимыми переменными.

Во-вторых, оценим коэффициенты регрессии каждой из упомянутых нами независимых переменных.

Нам нужна простая в интерпретации модель.

Одним из примеров простоты использования является то, что мы предпочитаем положительные коэффициенты для обозначения положительного качества. Это более высокая кредитоспособность.


Следовательно, в нашей переменной "good or bad" будет использоваться значение единицы для представления хороших счетов,

и мы будем использовать значение нуля для представления плохих или дефолтных учетных записей.


для этого используем np where- он проверяет, истинно ли условие, и присваивает соответствующие значения.
Воспользуемся методом isin. Этот метод проверяет, есть ли значение в списке значений.

In [53]:
Пример:
data['good or bad'] = np.where(data['loan_status'].isin(['Charged Off', 'Default',
                                                               'Does not meet the credit policy. Status:Charged Off',
                                                               'Late (31-120 days)']), 0, 1)

# Разделение данных, вес доказательств, информационная ценность

In [55]:
Импорт
from sklearn.model_selection import train_test_split

In [62]:
Разделяем данные на тренировочный и тестовый набор
data_inputs_train, data_inputs_test, data_targets_train, data_targets_test = train_test_split(data.drop('good or bad', axis = 1), data['good or bad'], test_size = 0.2, random_state = 42)

вес доказательств дискретной переменной равен натуральному логарифму отношения доли хороших заемщиков к

доле плохих заемщиков.

Поэтому, чтобы найти вес доказательств оценки, мы должны сначала найти соотношение хороших и плохих заемщиков

Начнем с количества заемщиков по каждому классу.

Для этого мы можем подсчитать строки, которые содержали каждую из оценок.

В данном случае делим по классам.

Еще одна информация, которая нам может понадобиться, - это соотношение хороших и плохих заемщиков в каждом из них.

Это можно суммировать либо по доле хороших заемщиков, либо по доле плохих заемщиков.

Неважно, какой из них, так как доля хороших заемщиков равна единице минус доля плохих заемщиков.

In [73]:
Пример
data

,grade,good_bad,grade,good_bad
0,A,15108,A,0.962338
1,B,27199,B,0.923085
2,C,25048,C,0.882905
3,D,15390,D,0.844314
4,E,7145,E,0.805178
5,F,2699,F,0.775472
6,G,668,G,0.697605


Во-вторых, рассчитываем пропорции хороших заемщиков и плохих заемщиков
процент хороших заемщиков и соответствующая категория класса равна количеству хороших заемщиков, которое делится на общее количество хороших заемщиков, которое представляет собой сумму хороших заемщиков во всех категориях.

Аналогичным образом, процент плохих заемщиков в соответствующей категории классов равен количеству плохих заемщиков, которое делится на общее количество плохих заемщиков.

In [79]:
data пример

,grade,n_obs,prop_good,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad
0,A,15108,0.962338,0.162004,14539.0,569.0,0.175027,0.055839
1,B,27199,0.923085,0.291656,25107.0,2092.0,0.302250,0.205299
2,C,25048,0.882905,0.268591,22115.0,2933.0,0.266231,0.287831
3,D,15390,0.844314,0.165028,12994.0,2396.0,0.156428,0.235132
4,E,7145,0.805178,0.076616,5753.0,1392.0,0.069257,0.136605
5,F,2699,0.775472,0.028942,2093.0,606.0,0.025197,0.059470
6,G,668,0.697605,0.007163,466.0,202.0,0.005610,0.019823


Теперь есть все необходимое для расчета веса доказательств по категориям оценок.

Он равен натуральному логарифму соотношения доли хороших заемщиков к доле плохих заемщиков.
data['W'] = np.log(data['prop_n_good'] / data['prop_n_bad'])
data

Также можно рассчитать информационную ценность.

Информационная ценность равна сумме доли добросовестных заемщиков по соответствующей категории минус доля безнадежных кредитов для соответствующей категории, умноженная на вес доказательств для соответствующей категории.

Столбец информационного значения одинаков в каждой строке, поскольку эта метрика ссылается на информационное значение переменной класса в целом.
data['IV'] = (data['prop_n_good'] - data['prop_n_bad']) * data['W']